# Set Up

In [2]:
import configparser
from datetime import datetime
import os
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from datetime import datetime

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

# Load all parquet files created earlier

In [5]:
# users
parquetFile = spark.read.parquet("users")
parquetFile.createOrReplaceTempView("users")
parquetFile.printSchema()

# songs
parquetFile = spark.read.parquet("songs")
parquetFile.createOrReplaceTempView("songs")
parquetFile.printSchema()

# artists
parquetFile = spark.read.parquet("artists")
parquetFile.createOrReplaceTempView("artists")
parquetFile.printSchema()

# time 
parquetFile = spark.read.parquet("times")
parquetFile.createOrReplaceTempView("times")
parquetFile.printSchema()

# songplays
parquetFile = spark.read.parquet("songplays")
parquetFile.createOrReplaceTempView("songplays")
parquetFile.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- lattitude: double (nullable = true)
 |-- longitude: double (nullable = true)

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

root
 |-- songplay_id: long (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nul

# Query

In [6]:
spark.sql("SELECT * FROM users").show(5)

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     88|  Mohammad|Rodriguez|     M| free|
|     75|    Joseph|Gutierrez|     M| free|
|     29|Jacqueline|    Lynch|     F| free|
|     68|    Jordan|Rodriguez|     F| free|
|     69|  Anabelle|  Simpson|     F| free|
+-------+----------+---------+------+-----+
only showing top 5 rows



In [7]:
spark.sql("SELECT * FROM songs WHERE year = 1969").show()

+------------------+---------+---------+----+------------------+
|           song_id|    title| duration|year|         artist_id|
+------------------+---------+---------+----+------------------+
|SOCIWDW12A8C13D406|Soul Deep|148.03546|1969|ARMJAGH1187FB546F3|
+------------------+---------+---------+----+------------------+



In [8]:
spark.sql("SELECT * FROM artists").show(5)

+------------------+--------------------+--------------------+---------+---------+
|         artist_id|                name|            location|lattitude|longitude|
+------------------+--------------------+--------------------+---------+---------+
|ARDR4AC1187FB371A1|Montserrat Caball...|                    |     null|     null|
|ARMAC4T1187FB3FA4C|The Dillinger Esc...|   Morris Plains, NJ| 40.82624|-74.47995|
|AREBBGV1187FB523D2|Mike Jones (Featu...|         Houston, TX|     null|     null|
|ARD842G1187B997376|          Blue Rodeo|Toronto, Ontario,...| 43.64856|-79.38533|
|AR9AWNF1187B9AB0B4|Kenny G featuring...|Seattle, Washingt...|     null|     null|
+------------------+--------------------+--------------------+---------+---------+
only showing top 5 rows



In [9]:
spark.sql("SELECT * FROM times").show(5)

+--------------------+----+---+----+-------+----+-----+
|          start_time|hour|day|week|weekday|year|month|
+--------------------+----+---+----+-------+----+-----+
|2018-11-15 11:15:...|  11| 15|  46|      5|2018|   11|
|2018-11-15 17:42:...|  17| 15|  46|      5|2018|   11|
|2018-11-15 19:13:...|  19| 15|  46|      5|2018|   11|
|2018-11-15 22:35:...|  22| 15|  46|      5|2018|   11|
|2018-11-21 01:48:...|   1| 21|  47|      4|2018|   11|
+--------------------+----+---+----+-------+----+-----+
only showing top 5 rows



In [10]:
spark.sql("SELECT * FROM songplays").show(1)

+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|songplay_id|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|year|month|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|          0|2018-11-21 21:56:...|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|2018|   11|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+



## Number of female users in the data set?

In [30]:
spark.sql("SELECT COUNT(DISTINCT(user_id)) AS num_Female_users \
        FROM users \
        WHERE gender = 'F'").show()

+----------------+
|num_Female_users|
+----------------+
|              55|
+----------------+



## Number of songs were played from the most played artist?

In [ ]:
# does it matter a.artist_id or s.artist_id???

In [31]:
spark.sql("SELECT a.name, COUNT(s.song_id) AS num_songs \
        FROM artists AS a \
        JOIN songs AS s \
        ON a.artist_id = s.artist_id \
        GROUP BY a.artist_id, a.name \
        ORDER BY num_songs DESC \
        ").show(5)

+--------------------+---------+
|                name|num_songs|
+--------------------+---------+
|                 Clp|        2|
|              Casual|        2|
|Mike Jones (Featu...|        1|
|           Andy Andy|        1|
|Kenny G featuring...|        1|
+--------------------+---------+
only showing top 5 rows



## Average number of hour users spent on listening song 

In [28]:
spark.sql("SELECT AVG(hour) AS avg_hour_spent FROM \
    (SELECT u.user_id, t.hour \
        FROM songplays AS ss \
        JOIN times AS t \
        ON ss.start_time = t.start_time \
        JOIN users AS u \
        ON ss.user_id = u.user_id \
        GROUP BY u.user_id, t.hour )\
        ").show(5)

+--------------+
|avg_hour_spent|
+--------------+
|          21.0|
+--------------+



## Song title that were played most

In [29]:
spark.sql("SELECT s.title, COUNT(s.song_id) AS num_play \
        FROM songs AS s \
        JOIN songplays AS ss \
        ON ss.song_id = s.song_id \
          GROUP BY ss.song_id, s.title \
        ").show()

+--------------+--------+
|         title|num_play|
+--------------+--------+
|Setanta matins|       1|
+--------------+--------+

